In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

In [22]:
data = pd.read_csv("C:/Users/anike/Downloads/100_Sales.csv")
data.head()

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit,Unnamed: 9,Unnamed: 10
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,27/06/2010,159.42,2533654.00,951410.50,NaN,NaN
1,Central America and the Caribbean,Grenada,Cereal,Online,C,15/09/2012,117.11,576782.80,248406.36,NaN,NaN
2,Europe,Russia,Office Supplies,Offline,L,05/08/2014,524.96,1158502.59,224598.75,NaN,NaN
3,Sub_Saharan Africa,Sao Tome and Principe,Fruits,Online,C,07/05/2014,6.92,75591.66,19525.82,NaN,NaN
4,Sub_Saharan Africa,Rwanda,Office Supplies,Offline,L,02/06/2013,524.96,3296425.02,639077.50,NaN,NaN


In [23]:
data.tail()

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit,Unnamed: 9,Unnamed: 10
95,Sub_Saharan Africa,Mali,Clothes,Online,M,09/03/2011,35.84,97040.64,65214.72,NaN,NaN
96,Asia,Malaysia,Fruits,Offline,L,28/12/2011,6.92,58471.11,15103.47,NaN,NaN
97,Sub_Saharan Africa,Sierra Leone,Vegetables,Offline,C,29/06/2016,90.93,228779.10,93748.05,NaN,NaN
98,North America,Mexico,Personal Care,Offline,M,08/08/2015,56.67,471336.91,144521.02,NaN,NaN
99,Sub_Saharan Africa,Mozambique,Household,Offline,L,15/02/2012,502.54,3586605.09,889472.91,NaN,NaN


In [15]:
df.describe()

,Unit_Cost,Total_Revenue,Total_Profit,Unnamed: 9,Unnamed: 10
count,100.000000,1.000000e+02,1.000000e+02,0.0,0.0
mean,191.048000,1.373488e+06,4.416820e+05,NaN,NaN
std,188.208181,1.460029e+06,4.385379e+05,NaN,NaN
min,6.920000,4.870260e+03,1.258020e+03,NaN,NaN
25%,35.840000,2.687212e+05,1.214436e+05,NaN,NaN
50%,107.275000,7.523144e+05,2.907680e+05,NaN,NaN
75%,263.330000,2.212045e+06,6.358288e+05,NaN,NaN
max,524.960000,5.997055e+06,1.719922e+06,NaN,NaN


In [16]:
df.shape

(100, 11)

In [17]:
df.isna().sum()

Region              0
Country             0
Item_Type           0
Sales_Channel       0
Order_Priority      0
Ship_Date           0
Unit_Cost           0
Total_Revenue       0
Total_Profit        0
Unnamed: 9        100
Unnamed: 10       100
dtype: int64

In [24]:
# Feature engineering
data['Profit_Margin'] = data['Total_Profit'] / data['Total_Revenue']
data['Revenue_per_Unit'] = data['Total_Revenue'] / data['Unit_Cost']

# Prepare the features (X) and the target (y)
X = data[['Unit_Cost', 'Total_Revenue', 'Profit_Margin', 'Revenue_per_Unit']]
y = data['Total_Profit']

# Log transform the target variable to handle skewed distributions
y_log = np.log1p(y)  # log1p applies log(1 + y) to avoid issues with 0 profits

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

In [25]:
# Define a function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Fit the model on the training data
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return mse, rmse

In [26]:
# Polynomial features with regularized models
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y_log, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

In [27]:
#Lasso with polynomial features
lasso_poly = make_pipeline(StandardScaler(), Lasso(random_state=42))
lasso_params = {'lasso__alpha': np.logspace(-6, 6, 13)}
lasso_grid = GridSearchCV(lasso_poly, lasso_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
lasso_grid.fit(X_train_poly, y_train)
best_lasso = lasso_grid.best_estimator_
lasso_mse, lasso_rmse = evaluate_model(best_lasso, X_train_poly, X_test_poly, y_train, y_test)

In [29]:
### Print results
print("Lasso Regression (Poly) - MSE: {:.6f}, RMSE: {:.6f}".format(lasso_mse, lasso_rmse))

Lasso Regression (Poly) - MSE: 0.189604, RMSE: 0.435435


In [30]:
### Print best hyperparameters
print("Best Lasso alpha:", best_lasso.named_steps['lasso'].alpha)

Best Lasso alpha: 0.001
